In [1]:
### general cleaning
import numpy as np
import pandas as pd

### Used to pull script from url
from bs4 import BeautifulSoup
import requests
import json

pd.set_option('display.max_colwidth', -1)
pd.set_option("display.max_rows",1500)

In [2]:
### url of godfather 1
# url_gf1 = "http://www.thegodfathertrilogy.com/gf1/transcript/gf1transcript.html"
# page_source = requests.get(url_gf1) ### get page's contents
# page_soup = BeautifulSoup(page_source.content, 'html.parser')

In [3]:
# ### goes through the html contents and finds blockquote markers that indicate script text
# text_block = page_soup.findAll('blockquote', class_ = '')
# text_lst = []
# for item in text_block:
#     text_lst = item.text.split('\n')

# text_lst = [x for x in text_lst if x != ''] #gets rid of empty string elements

# import json
# with open('data.json', 'w') as f:
#     json.dump(text_lst, f)

In [4]:
text_lst = list(pd.read_json('data.json')[0])

In [5]:
### source has all names in uppercase when speaking except Fabrizio so for loop makes it simpler
for index in range(len(text_lst)):
    quote = text_lst[index]
    if ('Fabrizio' == quote.strip()) | ('Fabrizio (' in quote):
        text_lst[index] = 'Fabrizio'.upper() + quote.strip()[8:]
        if 'translates' in text_lst[index]:
            text_lst[index] = text_lst[index][:text_lst[index].find(' ')] + ' (translates)'
        elif '(' in text_lst[index]:
            text_lst[index] = text_lst[index][:text_lst[index].find(' ')]
    
    if ('CAPTAIN' in quote):
        text_lst[index] = text_lst[index].replace('CAPTAIN', '')
                     
    if ('MURDERED' in quote)| ('EPIC' in quote) | ('SAGA' in quote)|\
        ('TRILOGY' in quote): # changes headline to lowercase to keep it part of text
        text_lst[index] = text_lst[index].lower()

In [6]:
transcript_lst = text_lst[1:-14] #starting from 1 to -14 w/ \n in split
joined = ' '.join(transcript_lst) #spaces between joined parts of arrays
joined = joined.replace('  ', ' ') #replaces double space with single

split_joined = joined.split()

In [7]:
all_name_split = []
string_line = ''
string_name = ''
i=0
while i in range(len(split_joined)-1):
    item = split_joined[i]

    if ((item.isupper() == True) or (item=='McCLUSKEY')) & (item!='I') \
        & (item!='I,') & (item!='--I') & (item!='A') & (item!='J') \
        & (item!='FBI') & ('NO!' not in item) & ('G.I.' not in item) \
        & ('\"I' not in item) & ('[A' not in item) & (item!='TRILOGY') \
        & (item.find('SAGA') == -1) & (item!='&') & ('OS' not in item) \
        & (item!= 'E') & (item!='E.') & (item!='L.A') & (item!='SCRAM!') \
        & (item!='WOP') & (item!='J.'):
        all_name_split.append(string_line)
        string_name = string_name + ' '+ item
        string_line = ''

    else:
        string_line = string_line + ' ' + item
        all_name_split.append(string_name)
        string_name = ''
    i+=1
    if (i == (len(split_joined))-1):
            all_name_split.append(string_line)
            all_name_split.append(item)

all_name_split = [x for x in all_name_split if x != '']

In [8]:
name_dialogue = []
i = 0

df = pd.DataFrame(columns = ['Role', 'Line'])
while i in range(len(all_name_split)-1):
    name_dialogue.append(all_name_split[i]+all_name_split[i+1])
    if 'Transcribed' in all_name_split[i+1]:
        temp = all_name_split[i+1]
        all_name_split[i+1] = temp[:temp.find('Transcribed')]
    elif '{' in all_name_split[i+1]:
        temp = all_name_split[i+1]
        all_name_split[i+1] = temp[:temp.find('{')]
    elif 'educational' in all_name_split[i+1]:
        temp = all_name_split[i+1]
        all_name_split[i+1] = temp[:temp.find('This')]
    df= df.append({'Role':all_name_split[i],'Line':all_name_split[i+1] }, ignore_index= True)

    i+=2

df= df.append({'Role':'THE END', 'Line':''}, ignore_index= True)

In [9]:
new_quote = []
for index,quote in enumerate(df['Line']):
        if '[In Italian: ' in quote:
            bracket_start = quote.find('[In Italian: ')
            bracket_end = quote.find(']', bracket_start+ 13)
            quote = quote[:bracket_start] + quote[bracket_start+13:bracket_end] + quote[bracket_end+1:]
            
        if '[In Latin, ' in quote:
            bracket_start = quote.find('[In Latin, ')
            bracket_end = quote.find(']', bracket_start+ 11)
            quote = quote[:bracket_start] + quote[bracket_start+11:bracket_end] + quote[bracket_end+1:]
  
        if 'in Italian: ' in quote:
            bracket_start = quote.find('in Italian: ')
            bracket_end = quote.find(']', bracket_start+ 12)
            quote = quote[bracket_start+12:bracket_end]   
            
        if index == 440:  ## without this gets stuck in infinite loop due to source input error
            bracket_start = quote.find('(')
            bracket_end = quote.find(']', bracket_start)+1
            quote = quote[0:bracket_start] + quote[bracket_end:]

        if index == 1213:  ## without this gets stuck in infinite loop due to source input error
            bracket_start = quote.find('[')
            bracket_end = quote.find(')', bracket_start)+1
            quote = quote[0:bracket_start] + quote[bracket_end:]    
            
        while (('(' in quote) and ('translates' not in quote)):
            bracket_start = quote.find('(')
            bracket_end = quote.find(')', bracket_start)+1
            quote = quote[0:bracket_start] + quote[bracket_end:]

        while ('[' in quote):
            bracket_start = quote.find('[')
            bracket_end = quote.find(']', bracket_start)+1
            quote = quote[0:bracket_start] + quote[bracket_end:]
            
        if ('***' in quote):
            bracket_start = quote.find('***')
            bracket_end = quote.find('***', bracket_start+3)
            quote = quote[0:bracket_start] + quote[bracket_end:]
            if ('***' in quote):
                quote = quote.replace('***', '')
                
        if index==1267:
            bracket_start = quote.find('...')+3
            quote = quote[0:bracket_start]
            
        if '-dawn' in quote: #for Woltz
            quote = quote.replace('-dawn', '')
        
        new_quote.append(quote.strip())

In [10]:
new_name = []
for name in df['Role']:
    if 'S VOICE' in name.upper():
        name = name[:name.find("\'")]
    
    if 'CORLEONE' in name.upper():
        if '\'' not in name:
            name = name[:-9]
            
    if "SITE" in name:
        name = name[name.find("SITE")+4:]
    
    if "SEQUENCE" in name:
        name = name[name.find("SEQUENCE")+9:]
    
    if "DISSOLVE" in name:
        name = name[name.find("DISSOLVE"):]
        
        
    ####### naming clarifications and/or multiple instances of same person changed to same string    
    if "TATTAGLIA" == name.strip(): ##gives better clarification to which member of the family
        name = "PHILIP" + name
    
    if "SCREAMS" == name.strip(): ##script only says screams not who it was
        name = "WOLTZ"
        
    if "GREEN" in name:
        name = name.replace('GREEN', '')
        
    if "HAGEN" in name:
        name = "TOM"
    
    if "BRASI" in name: 
        name = "LUCA"
    
    if "FRANK"== name.strip(): 
        name = "FRANKIE"
    
    if "PAN" in name.strip(): 
        name = "PAN:"
        
    if "OVER" in name.strip(): 
        name = name+":"
        
    new_name.append(name.strip())

In [11]:
df['Role'] = new_name
df['Line'] = new_quote

In [12]:
to_drop = [index for index, x in enumerate(df['Role']) if ('NOT TO' in x) | ('COPYRIGHT' in x)]
df = df.drop(index = to_drop).reset_index(drop=True)

In [13]:
with open('cleaned_godfather.json', 'w') as myfile:
    myfile.write(df.to_json(orient = 'values'))
    myfile.close()